In [88]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [6]:
import numpy as np
import cv2
import matplotlib.pyplot as plt
import pandas as pd
from tqdm import tqdm

In [7]:
df = pd.read_csv("/kaggle/input/fer2013-dataset/fer2013.csv")


matrix = np.zeros((48, 48), dtype=np.uint8)

train_images = []
test_images = []
train_labels = []
test_labels = []

data_type = df["Usage"].value_counts()
data_type = data_type.to_dict()
data_type["Test"] = data_type["PrivateTest"]+data_type["PublicTest"]
del data_type["PrivateTest"]
del data_type["PublicTest"]
print(data_type)

def atoi(s):
  n = 0
  for i in s:
    n = n * 10 + ord(i) - ord("0")
  return n


for value in tqdm(range(len(df))):
  pixels = df['pixels'][value]
  pixels_split = pixels.split()

  for values in range(2304):
    x_index = values // 48
    y_index = values % 48
    matrix[x_index][y_index] = atoi(pixels_split[values])

  if value<=data_type['Training']:
    train_images.append(matrix)
    train_labels.append(df["emotion"][value])
  else:
    test_images.append(matrix)
    test_labels.append(df["emotion"][value])

train_images = np.array(train_images)
test_images = np.array(test_images)
train_labels = np.array(train_labels)
test_labels = np.array(test_labels)
print(train_images)
print(test_images)

In [8]:
class conv:
    def __init__(self, number_of_filters, filter_dim):
        self.number_of_filters = number_of_filters
        self.filter_dim = filter_dim
        self.conv_filters = np.random.randn(number_of_filters, filter_dim, filter_dim)/(filter_dim * filter_dim)

    def image_region(self, image):
        height, width = image.shape
        self.image = image
        for j in range(height - self.filter_dim + 1):
            for k in range(width - self.filter_dim + 1):
                image_patch = image[ j : (j + self.filter_dim), k : (k + self.filter_dim)]
                yield image_patch, j, k

    def forward_propagation(self, image):
        height, width = image.shape
        conv_out = np.zeros((height - self.filter_dim + 1, width - self.filter_dim + 1, self.number_of_filters))
        for image_patch, i, j in self.image_region(image):
            conv_out[i, j] = np.sum(image_patch * self.conv_filters, axis = (1,2))
        return conv_out

    def back_propagation(self, dl_dout, learning_rate):
        dl_df_params = np.zeros(self.conv_filters.shape)
        for image_patch, i, j in self.image_region(self.image):
            for k in range(self.number_of_filters):
                dl_df_params[k] += image_patch*dl_dout[i, j, k]

            self.conv_filters -= learning_rate*dl_df_params
            return dl_df_params

In [9]:
class max_pool:
    def __init__(self, filter_dim):
        self.filter_dim = filter_dim

    def image_region(self, image):
        new_height = image.shape[0] // self.filter_dim
        new_width = image.shape[1] // self.filter_dim
        self.image = image

        for i in range(new_height):
            for j in range(new_width):
                image_patch = image[(i*self.filter_dim) :(i*self.filter_dim + self.filter_dim), (j*self.filter_dim) : (j*self.filter_dim + self.filter_dim)]
                yield image_patch, i, j

    def forward_propagation(self, image):
        height, width, number_of_filters = image.shape
        output = np.zeros((height // self.filter_dim, width // self.filter_dim, number_of_filters))

        for image_patch, i, j in self.image_region(image):
            output[i, j] = np.amax(image_patch, axis = (0,1))

        return output

    def back_propagation(self, dl_dout):
        dl_dmax_pool = np.zeros(self.image.shape)
        for image_patch, i, j in self.image_region(self.image):
            height, width, number_of_filters = image_patch.shape
            maximum_val = np.amax(image_patch, axis=(0,1))

            for i1 in range(height):
                for j1 in range(width):
                    for k1 in range(number_of_filters):
                        if image_patch[i1, j1, k1] == maximum_val[k1]:
                            dl_dmax_pool[i*self.filter_dim + i1, j*self.filter_dim + j1, k1] = dl_dout[i, j, k1]

            return dl_dmax_pool

In [10]:
class softmax:
    def __init__(self, input_node, softmax_node):
        self.weight = np.random.randn(input_node, softmax_node)/input_node
        self.bias = np.zeros(softmax_node)

    def forward_propagation(self, image):
        self.orig_im_shape = image.shape
        image_modified = image.flatten()
        self.modified_input = image_modified
        output_val = np.dot(image_modified, self.weight) + self.bias
        self.out = output_val
        exp_out = np.exp(output_val)

        return exp_out/np.sum(exp_out, axis = 0)

    def back_propagation(self, dl_dout, learning_rate):
        for i, grad in enumerate(dl_dout):
            if grad == 0:
                continue

            transformation_eq = np.exp(self.out)
            s_total = np.sum(transformation_eq)

            dy_dz = -transformation_eq[i]*transformation_eq / (s_total ** 2)
            dy_dz[i] = transformation_eq[i] * (s_total - transformation_eq[i]) / (s_total ** 2)

            dz_dw = self.modified_input
            dz_db = 1
            dz_d_inp = self.weight

            dl_dz = grad * dy_dz

            dl_dw = dz_dw[np.newaxis].T @ dl_dz[np.newaxis]
            dl_db = dl_dz * dz_db
            dl_d_inp = dz_d_inp @ dl_dz

        self.weight -= learning_rate * dl_dw
        self.bias -= learning_rate * dl_db

        return dl_d_inp.reshape(self.orig_im_shape)

In [11]:
convo = conv(10, 7)
pool = max_pool(4)
soft_max = softmax(10*10*10, 7)

In [12]:
def cnn_forward_prop(image, label):
    out_p = convo.forward_propagation((image / 255) - 0.5)
    out_p = pool.forward_propagation(out_p)
    out_p = soft_max.forward_propagation(out_p)

    cross_ent_loss = -np.log(out_p[label])
    accuracy_eval = 1 if np.argmax(out_p) == label else 0

    return out_p, cross_ent_loss, accuracy_eval

def training_cnn(image, label, learn_rate=0.001):

    out, loss, acc = cnn_forward_prop(image, label)

    gradient = np.zeros(7)
    gradient[label] = -1 / out[label]

    grad_back = soft_max.back_propagation(gradient, learn_rate)
    grad_back = pool.back_propagation(grad_back)
    grad_back = convo.back_propagation(grad_back, learn_rate)

    return loss, acc

In [13]:
for epoch1 in range(6):
    print('Epoch %d ----> ' %(epoch1 + 1))

    shuffle_data = np.random.permutation(len(train_images))
    train_images = train_images[shuffle_data]
    train_labels = train_labels[shuffle_data]

    loss = 0
    num_correct = 0

    for i, (im, label) in enumerate(zip(train_images, train_labels)):
        if i % 100 == 0:
            print("%d steps out of 100 steps: Average Loss %.3f and Accuracy: %d%%" %(i+1, loss / 100, num_correct))
            loss = 0
            num_correct = 0

        l1, accu = training_cnn(im, label)
        loss += l1
        num_correct += accu

print("Testing Phase")
loss = 0
num_correct = 0

for im, label in zip(test_images, test_labels):
    _, l1, accu = cnn_forward_prop(im, label)
    loss += l1
    num_correct += accu

num_tests = len(test_images)
print('Test Loss:', loss / num_tests)
print('Test Accuracy:', num_correct / num_tests)

In [35]:
print(test_images)
print(test_labels)

In [85]:
def predict(x):
    new_image = []
    loss_array = []
    accuracy_array = []
    data_dict = dict()
    emotionss = {0: 'Angry', 1: 'Disgust', 2: 'Fear', 3: 'Happy', 4: 'Neutral', 5: 'Sad', 6: 'Surprise'}
    for data in lab:
        new_image.append(x[10000])
    print(new_image)
    for im, label in zip(new_image, emotionss.keys()):
        loss = 0
        num_correct = 0
        _, l1, accu = cnn_forward_prop(im, label)
        loss += l1
        num_correct += accu
        loss_array.append(loss / 100)
        accuracy_array.append(num_correct)
        data_dict[label] = num_correct
        
        
    print(accuracy_array)
    print(data_dict)
    print(emotionss[max(data_dict, key=data_dict.get)])
    
predict(train_images)


In [78]:
"""
def predict(x):
    new_image = []
    loss_array = []
    accuracy_array = []
    new_images = []
    data_dict = dict()
    lab = [0, 1, 2, 3, 4, 5, 6]
    emotionss = {0: 'Angry', 1: 'Disgust', 2: 'Fear', 3: 'Happy', 4: 'Neutral', 5: 'Sad', 6: 'Surprise'}
    for data in lab:
        new_image.append(x[500])
    print(new_image)
    for im, label in zip(new_image, lab):
        loss = 0
        num_correct = 0
        out, l1, accu = cnn_forward_prop(im, label)
        loss += l1
        num_correct += accu
        
    print(out)
    for data in range(len(out)):
        value = lab[data]
        data_dict[value] = out[data]  
    print(data_dict)
    print(emotionss[max(data_dict, key=data_dict.get)])
    
    
predict(test_images)
"""

In [89]:
from keras.preprocessing import image

img = image.load_img("/kaggle/input/sample-image/im0.png",target_size = (48,48),color_mode = "grayscale")
img = np.array(img)
plt.imshow(img)
print(img.shape)

In [94]:
img = np.expand_dims(img,axis = 0) #makes image shape (1,48,48)
img = img.reshape(1,48,48)
img

In [96]:
def predict(x):
    new_image = []
    loss_array = []
    accuracy_array = []
    data_dict = dict()
    emotionss = {0: 'Angry', 1: 'Disgust', 2: 'Fear', 3: 'Happy', 4: 'Neutral', 5: 'Sad', 6: 'Surprise'}
    for data in emotionss.keys():
        new_image.append(x[0])
    print(new_image)
    for im, label in zip(new_image, emotionss.keys()):
        loss = 0
        num_correct = 0
        _, l1, accu = cnn_forward_prop(im, label)
        loss += l1
        num_correct += accu
        loss_array.append(loss / 100)
        accuracy_array.append(num_correct)
        data_dict[label] = num_correct
        
        
    print(accuracy_array)
    print(data_dict)
    print(emotionss[max(data_dict, key=data_dict.get)])
    
predict(img)